First List out the states to which the district belongs.

| Location   | State            |  
|------------|----------------|  
| NABHA      | Punjab         |  
| JAMMU WEST | Jammu & Kashmir          |  
| KATHLAL    | Gujarat       |  
| SAONGHAT   | Uttar Pradesh |  

Identify the right links

In [ ]:
import pandas as pd
import requests               # to get the webpage
from bs4 import BeautifulSoup # to parse the webpage


pages_left = True
root_url = "https://jivraj-18.github.io/tds-jan-2025-mock-roe-1/"
root_response = requests.get(root_url)
soup = BeautifulSoup(root_response.content, "html.parser")
data = []

page_count = 0

while pages_left:
  page_count += 1
  print(f"Page {page_count}")

  link = soup.find('a', string="Next page")
  next_page_url = link["href"] if link else None

  # Find the table by its class
  table = soup.find("div", class_="t")

  # Extract rows
  rows = table.find_all("div", class_="r")

  # Process data
  for row in rows:
      cols = [col.text.strip() for col in row.find_all("div", class_="c")]
      rowLink = row.select_one('a', href=True)
      if rowLink:
        rowLink = root_url + rowLink['href']
        cols.append(rowLink)
        data.append(cols)

  if link:
    # Replace with the actual URL
    url = root_url + next_page_url
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    print(f"Link {url}")

  if not link:
    pages_left=False

  # I want to find a specific state say Punjab and then save a list of the link in its link tags.
  # then using the list, scrap the links to get the tables I am getting here.


# Convert to DataFrame
cols=["Year", "State_Name", "Links"]
df = pd.DataFrame(data[1:], columns=cols)  # First row as header
df.head()

In [ ]:
districts = ['NABHA','JAMMU WEST', 'KATHLAL',	'SAONGHAT']

In [ ]:
# First List out the states to which the district belongs.
states_to_filter = ["PUNJAB", "JAMMU & KASHMIR", "GUJARAT", "UTTAR PRADESH"]

In [ ]:
# Filter the DataFrame
filtered_df = df[df["State_Name"].isin(states_to_filter)]

filtered_links = filtered_df['Links']

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

for link in filtered_links:
    url = link
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # print(soup.prettify())

    # Find the table by its class
    table = soup.find("div", class_="t")

    # Extract rows
    rows = table.find_all("div", class_="r")

    # Process data
    data = []
    for row in rows:
        cols = [col.text.strip() for col in row.find_all("div", class_="c")]
        data.append(cols)

# Convert to DataFrame
all_data = pd.DataFrame(data[1:], columns=data[0])  # First row as header
all_data.head()

,Unnamed: 0,ST_NAME,YEAR,AC,CANDIDATE,SEX,AGE,CATEGORY,PARTY,VOTES
0,425003,UTTAR PRADESH,2022,1 BEHAT,1 UMAR ALI KHAN,M,44.0,GEN,SP,134513
1,425004,UTTAR PRADESH,2022,1 BEHAT,2 NARESH SAINI,M,57.0,GEN,BJP,96633
2,425005,UTTAR PRADESH,2022,1 BEHAT,3 RAIS MALIK,M,54.0,GEN,BSP,45075
3,425006,UTTAR PRADESH,2022,1 BEHAT,4 POONAM KAMBOJ,F,36.0,GEN,INC,1632
4,425007,UTTAR PRADESH,2022,1 BEHAT,5 SUSHIL KUMAR,M,38.0,GEN,AAAP,1162


In [ ]:
all_data.drop(all_data.columns[0], axis=1, inplace=True)
all_data.drop_duplicates(inplace=True)

In [ ]:
all_data

,ST_NAME,YEAR,AC,CANDIDATE,SEX,AGE,CATEGORY,PARTY,VOTES
0,UTTAR PRADESH,2022,1 BEHAT,1 UMAR ALI KHAN,M,44.0,GEN,SP,134513
1,UTTAR PRADESH,2022,1 BEHAT,2 NARESH SAINI,M,57.0,GEN,BJP,96633
2,UTTAR PRADESH,2022,1 BEHAT,3 RAIS MALIK,M,54.0,GEN,BSP,45075
3,UTTAR PRADESH,2022,1 BEHAT,4 POONAM KAMBOJ,F,36.0,GEN,INC,1632
4,UTTAR PRADESH,2022,1 BEHAT,5 SUSHIL KUMAR,M,38.0,GEN,AAAP,1162
...,...,...,...,...,...,...,...,...,...
695,UTTAR PRADESH,2022,59 HAPUR,7 DR B P NILARATNA,M,69.0,SC,ASPKR,560
696,UTTAR PRADESH,2022,59 HAPUR,8 KIRAN PAL,M,40.0,SC,BMUP,471
697,UTTAR PRADESH,2022,59 HAPUR,9 KAILASHCHAND,M,55.0,SC,IND,335
698,UTTAR PRADESH,2022,59 HAPUR,10 VEERPAL SINGH,M,39.0,SC,AAAP,329


In [ ]:
all_data.to_csv('data.csv', index=False)

In [1]:
import pandas as pd

def clean_data(df):
    # Filter rows based on column: 'AC'
    df = df[df['AC'].str.contains("NABHA", regex=False, na=False, case=False)]
    # Drop column: 'CATEGORY'
    df = df.drop(columns=['CATEGORY'])
    # Drop column: 'AGE'
    df = df.drop(columns=['AGE'])
    # Retain top two votes per year
    df = df.groupby('YEAR', group_keys=False).head(2)
    # Convert VOTES column to numeric
    df['VOTES'] = pd.to_numeric(df['VOTES'], errors='coerce')
    # Calculate the margin
    df['MARGIN'] = df.groupby('YEAR')['VOTES'].diff(-1).abs()
    # Sort by column: 'MARGIN' (descending)
    df = df.sort_values(['MARGIN'], ascending=[False])
    return df

df = pd.read_csv('./data.csv')

df_clean = clean_data(df.copy())
df_clean.head()

,ST_NAME,YEAR,AC,CANDIDATE,SEX,PARTY,VOTES,MARGIN
15872,PUNJAB,1972,83 NABHA,1 GURDARSHAN SINGH,M,INC,35443,17538.0
19889,PUNJAB,2002,77 NABHA,1 RANDEEP SINGH,M,INC,37453,13951.0
14908,PUNJAB,1967,83 NABHA,1 N. SINGH,M,IND,24135,9506.0
13680,PUNJAB,1957,107 NABHA,1 BALWANT SINGH,M,INC,20475,8747.0
16439,PUNJAB,1977,77 NABHA,1 GURDARSHAN SINGH,M,INC,33644,5820.0
